In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import plotly.express as px
from sklearn.model_selection import train_test_split

# Normalización
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


# Métodos de Validación
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression


# Métricas para regresion
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_pickle(r'C:\Users\Dafne\OneDrive\Escritorio\hack a boss\proyecto_aviones\data\pickle\vuelos_limpio.pkl')

In [3]:
# Eliminamos las columnas que no aportan información extra y hora llegada real o retraso en la llegada para que no prediga el 100% de las veces y mes que solo tiene 1 valor
columnas_eliminar = [
    "latitude_destino",
    "latitude_origen",
    "longitude_destino",
    "longitude_origen",
    "codigo_aeropuerto_destino",
    "codigo_aeropuerto_origen",
    "direccion_destino",
    "direccion_origen",
    "retraso_llegada",
    "mes"
]
df.drop(columns=columnas_eliminar, inplace=True)


In [4]:
diccionario_costo = {
    'alto_costo': ["Delta Air Lines", "American Airlines", "United Airlines", "Alaska Airlines", "Hawaiian Airlines"],
    'medio_costo': ["Southwest Airlines", "JetBlue Airways"],
    'bajo_costo': ['Allegiant Air', 'Frontier Airlines', 'Spirit Airlines', 'Envoy Air', 'SkyWest Airlines', 'PSA Airlines', 
                'Endeavor Air', 'Mesa Airlines', 'Republic Airways', 'Horizon Air']}

def asignar_costo(aerolinea):
    for costo, aerolineas in diccionario_costo.items():
        if aerolinea in aerolineas:
            return costo

df['aerolinea'] = df['aerolinea'].apply(asignar_costo)

In [5]:
pesos_categorias = {'alto_costo': 2, 'medio_costo': 1, 'bajo_costo': 0} 

df['aerolinea'] = df['aerolinea'].map(pesos_categorias)

In [6]:
df["hora_salida_real"] = df["hora_salida_real"].apply(lambda x: x.hour * 60 + x.minute)
df["hora_llegada_real"] = df["hora_llegada_real"].apply(lambda x: x.hour * 60 + x.minute)
df["hora_salida_programada"] = df["hora_salida_programada"].apply(lambda x: x.hour * 60 + x.minute)

df['hora_despegue'] = df['hora_despegue'].replace('24:00', '00:00')
df['hora_despegue'] = pd.to_datetime(df['hora_despegue'], format='%H:%M').dt.hour * 60 + pd.to_datetime(df['hora_despegue'], format='%H:%M').dt.minute

In [7]:
# Como da error el datetime de fecha, extraemos el día del mes, que es el único que nos falta, porque mes como todos tienen el mismo no hace falta
# y sobbreescribimos la columna de fecha, 

df['fecha'] = df['fecha'].dt.day

In [8]:
def target_encoding(df, columna_categorica, target):

    media_columna_con_target = df.groupby(columna_categorica)[target].mean()
    df[columna_categorica] = df[columna_categorica].map(media_columna_con_target)
    return df

target_encoding(df, 'numero_cola', 'hora_llegada_real')
target_encoding(df, 'ciudad_origen', 'hora_llegada_real')
target_encoding(df, 'estado_origen', 'hora_llegada_real')
target_encoding(df, 'aeropuerto_origen', 'hora_llegada_real')
target_encoding(df, 'ciudad_destino', 'hora_llegada_real')
target_encoding(df, 'estado_destino', 'hora_llegada_real')
target_encoding(df, 'aeropuerto_destino', 'hora_llegada_real')

,aerolinea,fecha,numero_vuelo,numero_cola,hora_salida_programada,hora_salida_real,duracion_programada_vuelo,duracion_real,retraso_salida,hora_despegue,...,aeropuerto_origen,hora_llegada_real,dia_semana,anio,fin_de_semana,festivos,ciudad_destino,estado_destino,aeropuerto_destino,distancia_millas
0,0,1,3632,883.241597,491,496,57,69,5,513,...,751.880096,565,2,2021,0,0,840.014819,853.764107,840.014819,159.757805
1,0,1,3647,874.028269,405,397,56,61,-8,410,...,751.880096,458,2,2021,0,0,840.014819,853.764107,840.014819,159.757805
2,0,1,3723,899.018605,887,878,54,67,-9,891,...,751.880096,945,2,2021,0,0,840.014819,853.764107,840.014819,159.757805
3,0,1,4192,883.463158,755,755,61,59,0,764,...,751.880096,814,2,2021,0,0,840.014819,853.764107,840.014819,159.757805
4,0,1,4311,872.862559,1116,1187,62,58,71,1199,...,751.880096,1245,2,2021,0,0,840.014819,853.764107,840.014819,159.757805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1692471,0,30,4262,836.628676,988,1011,85,74,23,1029,...,718.446429,1085,5,2023,1,0,827.003715,829.039601,827.003715,255.813380
1692472,0,31,4244,754.419162,1051,1046,83,64,-5,1055,...,718.446429,1110,4,2021,0,0,827.003715,829.039601,827.003715,255.813380
1692473,0,31,4255,795.233184,335,393,84,80,58,423,...,718.446429,473,4,2021,0,0,827.003715,829.039601,827.003715,255.813380
1692474,0,31,4299,768.211765,420,420,78,70,0,438,...,718.446429,490,5,2022,1,0,827.003715,829.039601,827.003715,255.813380


In [9]:
variables_discretas= ['aerolinea', 'fecha', 'numero_vuelo', 'numero_cola', 'hora_salida_programada', 'hora_salida_real', 'ciudad_destino', 
                      'estado_destino', 'aeropuerto_destino',  'ciudad_origen', 'estado_origen','aeropuerto_origen']
variables_continuas= ['duracion_programada_vuelo', 'duracion_real', 'distancia_millas']

In [10]:
# Para variables discretas (tanto numéricas como categóricas, toman un conjunto finito o numerable de valores)

def comparar_variable_discreta_con_target(dataframe, target_variable, variable_to_compare):
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    sns.countplot(x=dataframe[variable_to_compare], ax=axes[0])
    sns.countplot(x=dataframe[variable_to_compare], hue=dataframe[target_variable], ax=axes[1])
    plt.show()

# comparar_variable_discreta_con_target(df, "hora_llegada_real", "aerolinea")

In [11]:
# Para variables continuas (numéricas que pueden contener valores atípicos, que pueden variar dentro de un rango específico )
def comparar_variable_continua_con_target(dataframe, target_variable, variable_to_compare):
    fig, axes = plt.subplots(2, 2, figsize=(18, 11))
    sns.boxplot(x=dataframe[variable_to_compare], ax=axes[0, 0])
    axes[0, 0].set_title(f'{variable_to_compare} Boxplot')
    sns.histplot(x=dataframe[variable_to_compare], ax=axes[0, 1])
    axes[0, 1].set_title(f'{variable_to_compare} Histogram')
    sns.boxplot(x=dataframe[target_variable], y=dataframe[variable_to_compare], ax=axes[1, 0])
    axes[1, 0].set_title(f'{variable_to_compare} vs {target_variable}')
    sns.histplot(x=dataframe[variable_to_compare], hue=dataframe[target_variable], ax=axes[1, 1])
    axes[1, 1].set_title(f'{variable_to_compare} Histogram by {target_variable}')
    plt.tight_layout()
    plt.show()

# comparar_variable_continua_con_target(df, "hora_llegada_real", "distancia_millas")

In [12]:
# NO CARGUEIS ESTO !!!
# os arrepentiréis xD

# comparar_variable_continua_con_target(df, "hora_llegada_real", "distancia_millas")

In [13]:
# VER SI QUEREMOS APLICAR CUARTILES

def calcular_cuartiles(df, variable_aplicar_cuartiles, n_inferior, n_superior):
    longitud_variable_antes = len(df[variable_aplicar_cuartiles])
    q1 = df[variable_aplicar_cuartiles].quantile(q=0.25)  # Q1
    q3 = df[variable_aplicar_cuartiles].quantile(q=0.75)  # Q3
    # Rango intercuartil (IQR)
    iqr = q3 - q1
    # Calcular los limites inferior y superior
    limite_inferior = q1 - n_inferior * iqr
    limite_superior = q3 + n_superior * iqr
    df = df[df[variable_aplicar_cuartiles].between(limite_inferior, limite_superior)]
    longitud_variable_despues = len(df[variable_aplicar_cuartiles])
    diferencia_filas = longitud_variable_antes - longitud_variable_despues
    print(f"La longitud de la variable antes era de {longitud_variable_antes}, y ahora es de {longitud_variable_despues}. Se han eliminado {diferencia_filas} filas del DF.")
    return df



In [14]:
# APLICO LOGARITMO

df["distancia_millas"] = np.log(df["distancia_millas"] + 1) 

In [15]:
# Definimos la target

TARGET = "hora_llegada_real"
X = df.drop(columns = TARGET)
y  = df[[TARGET]]

In [16]:
# Definimos train y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify = y)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (1353980, 26), y_train: (1353980, 1)
X_test: (338496, 26), y_test: (338496, 1)


In [17]:
x_scaler = StandardScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = y_scaler.transform(np.array(y_test).reshape(-1, 1))


Algunos algoritmos comunes utilizados en problemas de regresión incluyen la regresión lineal, regresión polinómica, regresión de vecinos más cercanos (KNN), regresión de bosques aleatorios, entre otros.

In [18]:
# INICIAMOS EL MODELO
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

yhat = model.predict(X_test)
yhat

c:\Users\Dafne\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


array([-1.96682281,  1.03241949, -0.37936699, ...,  1.34276307,
        0.64632164, -0.88305312])

In [19]:
y_test = y_scaler.inverse_transform(y_test)
yhat = y_scaler.inverse_transform(yhat.reshape(-1, 1))

In [20]:
def invertir_log (dato):

    return np.exp(dato) -1

df["distancia_millas"] = invertir_log(df["distancia_millas"])

In [21]:
r2 = r2_score(y_test, yhat)
mae = mean_absolute_error(y_test, yhat)
mse = mean_squared_error(y_test, yhat)
rmse = np.sqrt(mean_squared_error(y_test, yhat))
linear_model = [[str(model), model, r2, mae, mse, rmse]]
df_lr = pd.DataFrame(data = linear_model, columns = ["name", "model", "r2", "mae", "mse", "rmse"])
df_lr

,name,model,r2,mae,mse,rmse
0,RandomForestRegressor(),"(DecisionTreeRegressor(max_features=1.0, rando...",0.995057,1.696211,632.269332,25.144966


In [22]:
columna = TARGET

correlaciones = df.corr()[columna]

correlaciones_significativas = correlaciones[correlaciones > 0.2]

if len(correlaciones_significativas) > 0:
    print("Las siguientes columnas tienen una correlación mayor que 0.4 con '{}' :".format(columna))
    print(correlaciones_significativas.drop(columna).index.tolist())
else:
    print("No hay columnas con una correlación mayor que 0.4 con '{}'.".format(columna))

Las siguientes columnas tienen una correlación mayor que 0.4 con 'hora_llegada_real' :
['hora_salida_programada', 'hora_salida_real', 'hora_despegue']
